In [1]:
from datetime import datetime

from pytket import Circuit

import qnexus as qnx

from nexus_dataclasses.backend_config import AerConfig


In [21]:
#qnx.auth.login()

In [2]:
project_ref = qnx.projects.create(
    name=f"demo_{str(datetime.now())}",
    description="a demo project",
)

In [4]:
project_ref.summarize()

,name,description,id
0,demo_2024-04-23 17:15:23.138128,a demo project,b218fbec-0a78-4414-be5b-669cdc868bfd


In [4]:
project_ref.summarize()

,name,decription,id
0,demo_2024-04-22 15:22:57.323512,a demo project,f3601338-f4e2-4745-9c5b-425ca9f3e2ac


In [5]:
qnx.projects.add_property(
    name="Parameter_1",
    property_type="float",
    project=project_ref,
    description="Parameter for my iterative algorithm",
    required=False
)

In [5]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

In [6]:
my_circuit_ref = qnx.circuits.create(
    circuit=c, 
    project=project_ref,
    name="My circuit",
    #description="A new circuit",
    #properties={"Parameter_1": 0.1}
)

ResourceCreateFailed: Failed to create resource with status code: 422, message: {'detail': [{'type': 'missing', 'loc': ['body', 'data', 'attributes', 'name'], 'msg': 'Field required', 'input': {'bits': [['c', [0]], ['c', [1]]], 'commands': [{'args': [['q', [0]]], 'op': {'type': 'H'}}, {'args': [['q', [0]], ['q', [1]]], 'op': {'type': 'CX'}}, {'args': [['q', [0]], ['c', [0]]], 'op': {'type': 'Measure'}}, {'args': [['q', [1]], ['c', [1]]], 'op': {'type': 'Measure'}}], 'created_qubits': [], 'discarded_qubits': [], 'implicit_permutation': [[['q', [0]], ['q', [0]]], [['q', [1]], ['q', [1]]]], 'phase': '0.0', 'qubits': [['q', [0]], ['q', [1]]], 'properties': {}}, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]}

In [8]:
my_circuit_ref.summarize()

,name,decription,Parameter_1,project,id
0,My circuit,A new circuit,0.1,demo_2024-04-22 15:22:57.323512,7452e87e-58dd-46d8-8de5-5fb2594bc4c1


In [9]:
compile_job_ref = qnx.jobs.compile.run(
    name=f"compile_job_{str(datetime.now())}",
    circuits=my_circuit_ref,
    optimisation_level=1,
    project=project_ref,
    target=AerConfig()
)

In [10]:
qnx.jobs.filter(name="compile", project_id=project_ref.id).all().summarize()

,name,decription,job_type,last_status,project,id
0,compile-job-infundibuliform-terracotta-twinkli...,None,JobType.Compile,StatusEnum.COMPLETED,placeholder,5b636943-cb08-4168-ba94-a6eaa5ff6ebc
1,compile-job-adnate-death-reserved-hypothejus,None,JobType.Execute,StatusEnum.COMPLETED,placeholder,fc709079-9ea0-49c8-96c2-1e5acdd2d0dc
2,compile-job-irregular-st-inflated-alboviolaceus,None,JobType.Compile,StatusEnum.ERROR,placeholder,0acf01f6-15ae-4fc3-b78c-437c2ec7c2c8
3,compile-job-irregular-green-mischievous-vernum,None,JobType.Compile,StatusEnum.ERROR,placeholder,546d82dd-a7c1-4321-8145-6fb458a87225
4,compile-job-offset-sulphur-scampering-lacera,None,JobType.Compile,StatusEnum.ERROR,placeholder,03691cb7-6be5-4933-90da-1fde22f44bb4
...,...,...,...,...,...,...
198,bingo_compile,None,JobType.Compile,StatusEnum.COMPLETED,placeholder,bcf75689-bdee-47fd-8a6d-8120148f5e22
199,compile_job_VQE_2024-04-22 15:11:19.260246_0,None,JobType.Compile,StatusEnum.COMPLETED,placeholder,09724c70-eb37-4b85-b265-00af50a16d62
200,compile_job_VQE_2024-04-22 15:16:45.664260_0,None,JobType.Compile,StatusEnum.COMPLETED,placeholder,1250f41c-7c85-4ba4-afff-a56d73ef9842
201,compile_job_2024-04-22 15:20:25.850489,None,JobType.Compile,StatusEnum.COMPLETED,placeholder,978f4cff-8f48-474d-9501-cc676178d423


In [11]:
qnx.jobs.wait_for(compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 4, 22, 14, 23, 3, 547019, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 4, 22, 14, 23, 2, 859796, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [12]:
compiled_circuit_refs = qnx.jobs.compile.results(compile_job_ref)

compiled_circuit_refs[0].compiled_circuit

CircuitRef(id=UUID('85b5cbe9-3f46-45c1-a119-fad6c548ac19'), annotations=Annotations(name='My circuit-AerBackend-final', description=None, properties={}), project=ProjectRef(id=UUID('f3601338-f4e2-4745-9c5b-425ca9f3e2ac'), annotations=Annotations(name='demo_2024-04-22 15:22:57.323512', description='a demo project', properties={})))

In [13]:
execute_ref = qnx.jobs.execute.run(
    name="My Execute Job",
    circuits=[item.compiled_circuit for item in compiled_circuit_refs],
    project=project_ref,
    n_shots=[10],
    target=AerConfig()
)

In [14]:
await qnx.jobs.listen_job_status(execute_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 4, 22, 14, 23, 19, 395924, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 4, 22, 14, 23, 18, 973084, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [15]:
my_results = [item.backend_result for item in qnx.jobs.execute.results(execute_ref)]